# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,badger,64.8000,-147.5333,-16.07,72,100,0.00,US,1704904679
1,1,kargasok,59.0556,80.8572,-13.91,80,100,12.07,RU,1704904844
2,2,adamstown,-25.0660,-130.1015,24.03,87,70,3.91,PN,1704904461
3,3,qaqortoq,60.7167,-46.0333,0.81,90,100,2.42,GL,1704904847
4,4,sabang,5.8933,95.3214,27.60,78,60,5.67,ID,1704904370


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points("Lng",
                                        "Lat",
                                        geo = True,
                                        tiles = "OSM",
                                        frame_width = 700,
                                        frame_height = 500,
                                        size = "Humidity",
                                        color = "City"
                                        )

# Display the map
map_plot_1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 27)
                                    & (city_data_df["Max Temp"] > 21)
                                    & (city_data_df["Cloudiness"] < 10)
                                    & (city_data_df["Wind Speed"] < 4.5)
                                      ]

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,saipan,15.1355,145.7010,26.39,83,0,2.57,MP,1704904858
34,34,siem reap,13.5000,104.0000,24.91,68,9,1.11,KH,1704904892
143,143,tindouf,27.6711,-8.1474,23.25,15,0,3.09,DZ,1704905066
177,177,bardai,11.6500,7.2333,26.70,8,0,3.97,NG,1704905115
215,215,sanctuary point,-35.1114,150.6290,22.64,96,7,2.31,AU,1704905171
247,247,taghazout,30.5426,-9.7112,23.43,50,0,3.11,MA,1704905217
304,304,timimoun,29.2639,0.2310,21.21,14,0,1.54,DZ,1704905300


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
11,saipan,MP,15.1355,145.7010,83,
34,siem reap,KH,13.5000,104.0000,68,
143,tindouf,DZ,27.6711,-8.1474,15,
177,bardai,NG,11.6500,7.2333,8,
215,sanctuary point,AU,-35.1114,150.6290,96,
247,taghazout,MA,30.5426,-9.7112,50,
304,timimoun,DZ,29.2639,0.2310,14,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude= row["Lat"]
    longitude= row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address1 = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address1["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saipan - nearest hotel: Chalan Kanoa Beach Hotel
siem reap - nearest hotel: No hotel found
tindouf - nearest hotel: محمد بوسبي
bardai - nearest hotel: No hotel found
sanctuary point - nearest hotel: Dungowan Accommodation
taghazout - nearest hotel: Riad Taghazout
timimoun - nearest hotel: أناف


,City,Country,Lat,Lng,Humidity,Hotel Name
11,saipan,MP,15.1355,145.7010,83,Chalan Kanoa Beach Hotel
34,siem reap,KH,13.5000,104.0000,68,No hotel found
143,tindouf,DZ,27.6711,-8.1474,15,محمد بوسبي
177,bardai,NG,11.6500,7.2333,8,No hotel found
215,sanctuary point,AU,-35.1114,150.6290,96,Dungowan Accommodation
247,taghazout,MA,30.5426,-9.7112,50,Riad Taghazout
304,timimoun,DZ,29.2639,0.2310,14,أناف


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points("Lng", 
                                   "Lat",
                                   geo = True,
                                   color = "City",
                                   alpha = .6,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["City", "Country", "Hotel Name", "Humidity"],
                                   frame_width = 600, 
                                   frame_height = 400,
                                   aspect = 1,
                                  )

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)